Uladzislau Lukashevich 155671, Kiril Andrukh 162069


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import pandas as pd
from joblib import Parallel, delayed

from hamiltonian_cycle.algorithms.lab7 import LargeNeighborhoodSearch
from hamiltonian_cycle.costs import dm, function_cost
from hamiltonian_cycle.plots import plot_solution

In [ ]:
def read_dataset_csv(csv_path: Path) -> pd.DataFrame:
    return pd.read_csv(csv_path, sep=";", names=["x", "y", "cost"])


DATA_DIR = Path("../data").resolve()

ds_a = read_dataset_csv(DATA_DIR / "TSPA.csv")
ds_b = read_dataset_csv(DATA_DIR / "TSPB.csv")

dm_a = dm(ds_a)
dm_b = dm(ds_b)

# LNS

## Pseudocode

## Results with LS

### Dataset A

In [ ]:
class LNSMetrics:
    def __init__(self, solution: pd.DataFrame, num_iterations: float):
        self.cost = function_cost(solution)
        self.solution = list(solution.index)
        self.num_iterations = num_iterations


def run_lns(lns: LargeNeighborhoodSearch) -> LNSMetrics:
    lns_solution, num_iterations = lns()
    return LNSMetrics(lns_solution, num_iterations)

In [ ]:
max_runtime_sec = 2200
lns_a = LargeNeighborhoodSearch(
    ds=ds_a,
    dm=dm_a,
    max_runtime=max_runtime_sec,
    w_cost=0.5,
    w_regret=0.5,
    apply_local_search=True,
)
lns_runs = 20

metrics: list[LNSMetrics] = Parallel(n_jobs=-1)(
    delayed(run_lns)(lns_a) for _ in range(lns_runs)
)

minimum = min(metrics, key=lambda x: x.cost)
mean = sum([metric.cost for metric in metrics]) / len(metrics)
maximum = max(metrics, key=lambda x: x.cost)
mean_n_iterations = sum([metric.num_iterations for metric in metrics]) / len(metrics)

In [ ]:
print(f"Best solution: {minimum.solution}")
print("Objective function statistics:")
print(f"{minimum.cost = }\n{mean = }\n{maximum.cost= }")
print(f"Number of loop iterations: {mean_n_iterations}")
plot_solution(ds_a, minimum.solution, title="ILS")

### Dataset B

In [ ]:
max_runtime_sec = 2200
lns_b = LargeNeighborhoodSearch(
    ds=ds_b,
    dm=dm_b,
    max_runtime=max_runtime_sec,
    w_cost=0.5,
    w_regret=0.5,
    apply_local_search=True,
)
lns_runs = 20

metrics: list[LNSMetrics] = Parallel(n_jobs=-1)(
    delayed(run_lns)(lns_b) for _ in range(lns_runs)
)

minimum = min(metrics, key=lambda x: x.cost)
mean = sum([metric.cost for metric in metrics]) / len(metrics)
maximum = max(metrics, key=lambda x: x.cost)
mean_n_iterations = sum([metric.num_iterations for metric in metrics]) / len(metrics)

In [ ]:
print(f"Best solution: {minimum.solution}")
print("Objective function statistics:")
print(f"{minimum.cost = }\n{mean = }\n{maximum.cost= }")
print(f"Number of loop iterations: {mean_n_iterations}")
plot_solution(ds_b, minimum.solution, title="ILS")

## Results without LS

### Dataset A

In [ ]:
max_runtime_sec = 2200
lns_a = LargeNeighborhoodSearch(
    ds=ds_a,
    dm=dm_a,
    max_runtime=max_runtime_sec,
    w_cost=0.5,
    w_regret=0.5,
    apply_local_search=False,
)
lns_runs = 20

metrics: list[LNSMetrics] = Parallel(n_jobs=-1)(
    delayed(run_lns)(lns_a) for _ in range(lns_runs)
)

minimum = min(metrics, key=lambda x: x.cost)
mean = sum([metric.cost for metric in metrics]) / len(metrics)
maximum = max(metrics, key=lambda x: x.cost)
mean_n_iterations = sum([metric.num_iterations for metric in metrics]) / len(metrics)

In [ ]:
print(f"Best solution: {minimum.solution}")
print("Objective function statistics:")
print(f"{minimum.cost = }\n{mean = }\n{maximum.cost= }")
print(f"Number of loop iterations: {mean_n_iterations}")
plot_solution(ds_a, minimum.solution, title="ILS")

### Dataset B

In [ ]:
max_runtime_sec = 2200
lns_b = LargeNeighborhoodSearch(
    ds=ds_b,
    dm=dm_b,
    max_runtime=max_runtime_sec,
    w_cost=0.5,
    w_regret=0.5,
    apply_local_search=False,
)
lns_runs = 20

metrics: list[LNSMetrics] = Parallel(n_jobs=-1)(
    delayed(run_lns)(lns_b) for _ in range(lns_runs)
)

minimum = min(metrics, key=lambda x: x.cost)
mean = sum([metric.cost for metric in metrics]) / len(metrics)
maximum = max(metrics, key=lambda x: x.cost)
mean_n_iterations = sum([metric.num_iterations for metric in metrics]) / len(metrics)

In [ ]:
print(f"Best solution: {minimum.solution}")
print("Objective function statistics:")
print(f"{minimum.cost = }\n{mean = }\n{maximum.cost= }")
print(f"Number of loop iterationss: {mean_n_iterations}")
plot_solution(ds_b, minimum.solution, title="ILS")

# Summary

In [ ]:
a_res = pd.DataFrame(
    {
        "Steepest edge LS": [72046, 74033.715, 78801, 9.54, 1],
        "MLSM": [70662, 71267.4, 71693, 2223, 200],
        "ILS": [69107, 69326.15, 69765, 2223, 1106.2],
        "LNS with LS": [69474, 70179.05, 71022, 2223, 3027.2],
        "LNS without LS": [69657, 70494.5, 71452, 2223, 6705.1],
        "Greedy weighted cycle": [71057.0, 72218.320, 73587.0, 0.4, 1],
    },
    index=pd.MultiIndex.from_tuples(
        [
            ("Dataset A", "min"),
            ("Dataset A", "mean"),
            ("Dataset A", "max"),
            ("Dataset A", "seconds/instance"),
            ("Dataset A", "iterations"),
        ]
    ),
).T

b_res = pd.DataFrame(
    {
        "Steepest edge LS": [45393, 48264.78, 50697, 9.02, 1],
        "MLSM": [45321, 45751.25, 4613, 2218, 200],
        "ILS": [43493, 43783.05, 44312, 2218, 1114.8],
        "LNS with LS": [43568, 44290.55, 45011, 2218, 3058.25],
        "LNS without LS": [43595, 44507.05, 45558, 2218, 6717.85],
        "Greedy weighted cycle": [45453.0, 46252.105, 47884.0, 0.4, 1],
    },
    index=pd.MultiIndex.from_tuples(
        [
            ("Dataset B", "min"),
            ("Dataset B", "mean"),
            ("Dataset B", "max"),
            ("Dataset B", "seconds/instance"),
            ("Dataset B", "iterations"),
        ]
    ),
).T

a_res.join(b_res).sort_values(by=("Dataset A", "mean"))

# Conclusion

We didn't manage to achieve better results with LNS than we achieved with ILS. However, our results of LNS with and without LS are almost similar, meaning that 2 times more destroy and repair operations have +- same effect as polishing with local seacrh.
Nevertheless, LNS, despite having the same time budget as MLSM resulted in smaller costs.